# MLflow results presentation

In [1]:
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri('file:///home/zeyusun/work/flare-prediction-smarp/mlruns')
client = MlflowClient()

Display boarderlines of pandas tables

In [3]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

## Scripts

In [242]:
experiment_name = 'leaderboard1'
parent_run_name = 'sklearn_balanced'
tags = {
    'tags.database_name': 'database',
    'tags.dataset_name': 'dataset',
    #'tags.balanced': 'balanced',
    'tags.estimator_name': 'estimator',
}
metrics = {'metrics.' + m: new_m for m, new_m in [
    ['auc', 'auc'],
    ['tss_opt', 'tss'],
    #'tss',
    #'hss2',
    #'precision',
    #'recall',
]}
columns = {**tags, **metrics}

In [250]:
# Specify experiment
exp_id = client.get_experiment_by_name(experiment_name).experiment_id
runs = mlflow.search_runs(exp_id)
#runs = mlflow.search_runs(“<experiment_id>”, “metrics.loss < 2.5”)
#runs[['params.num_epochs','metrics.loss']].groupby("params.num_epochs").agg("min")

# Specify parent run
parentRunId = runs.loc[runs['tags.mlflow.runName'] == parent_run_name, 'run_id'].item()
runs = runs.loc[runs['tags.mlflow.parentRunId'] == parentRunId]

# Specify rows
runs = runs.loc[runs['status'] == 'FINISHED']
runs = runs.loc[runs['tags.balanced'] == 'True']
#runs = runs.loc[(runs['tags.estimator_name'] != 'HistGradientBoostingClassifier')]
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.tss,metrics.recall,metrics.hss1,metrics.auc,...,tags.mlflow.runName,tags.balanced,tags.mlflow.source.git.commit,tags.mlflow.parentRunId,tags.mlflow.log-model.history,tags.mlflow.source.type,tags.estimator_name,tags.mlflow.source.name,tags.logger_version,tags.checkpoint
3,774f450c11364e81bd792bc196da8e58,12,FINISHED,file:///home/zeyusun/work/flare-prediction-sma...,2021-05-25 23:07:41.295000+00:00,2021-05-25 23:07:49.492000+00:00,0.805970,0.820896,0.805970,0.988667,...,M_Q_6hr_combined_balanced_HistGradientBoosting...,True,a59f93c4de3e14b2a4037fbe9146138c6bdc1fd1,81d5aeb899f444c2998491ca6bff7ab9,"[{""run_id"": ""774f450c11364e81bd792bc196da8e58""...",LOCAL,HistGradientBoostingClassifier,run_sklearn.py,None,None
4,d081096a1f144591989d8f44281c3b1b,12,FINISHED,file:///home/zeyusun/work/flare-prediction-sma...,2021-05-25 23:07:23.216000+00:00,2021-05-25 23:07:41.285000+00:00,0.559701,0.574627,0.559701,0.973157,...,M_Q_6hr_combined_balanced_RandomForestClassifier,True,a59f93c4de3e14b2a4037fbe9146138c6bdc1fd1,81d5aeb899f444c2998491ca6bff7ab9,"[{""run_id"": ""d081096a1f144591989d8f44281c3b1b""...",LOCAL,RandomForestClassifier,run_sklearn.py,None,None
5,fe2ac762310e4b35817c732f90ba33f5,12,FINISHED,file:///home/zeyusun/work/flare-prediction-sma...,2021-05-25 23:07:20.376000+00:00,2021-05-25 23:07:23.207000+00:00,0.567164,0.574627,0.567164,0.980731,...,M_Q_6hr_combined_balanced_SGDClassifier,True,a59f93c4de3e14b2a4037fbe9146138c6bdc1fd1,81d5aeb899f444c2998491ca6bff7ab9,"[{""run_id"": ""fe2ac762310e4b35817c732f90ba33f5""...",LOCAL,SGDClassifier,run_sklearn.py,None,None
9,33991073986c4ac7926a4e5831700efc,12,FINISHED,file:///home/zeyusun/work/flare-prediction-sma...,2021-05-25 23:04:41.128000+00:00,2021-05-25 23:04:48.073000+00:00,0.858209,0.888060,0.858209,0.978976,...,M_Q_6hr_sharp_balanced_HistGradientBoostingCla...,True,a59f93c4de3e14b2a4037fbe9146138c6bdc1fd1,81d5aeb899f444c2998491ca6bff7ab9,"[{""run_id"": ""33991073986c4ac7926a4e5831700efc""...",LOCAL,HistGradientBoostingClassifier,run_sklearn.py,None,None
10,aa63535215824812bd9652bd64d34795,12,FINISHED,file:///home/zeyusun/work/flare-prediction-sma...,2021-05-25 23:04:31.339000+00:00,2021-05-25 23:04:41.118000+00:00,0.850746,0.888060,0.850746,0.974187,...,M_Q_6hr_sharp_balanced_RandomForestClassifier,True,a59f93c4de3e14b2a4037fbe9146138c6bdc1fd1,81d5aeb899f444c2998491ca6bff7ab9,"[{""run_id"": ""aa63535215824812bd9652bd64d34795""...",LOCAL,RandomForestClassifier,run_sklearn.py,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,7c8b59299df949e8997049ced228ce46,12,FINISHED,file:///home/zeyusun/work/flare-prediction-sma...,2021-05-25 19:57:13.739000+00:00,2021-05-25 19:57:25.265000+00:00,0.831933,0.924370,0.831933,0.976343,...,M_QS_6hr_sharp_balanced_RandomForestClassifier,True,a59f93c4de3e14b2a4037fbe9146138c6bdc1fd1,81d5aeb899f444c2998491ca6bff7ab9,"[{""run_id"": ""7c8b59299df949e8997049ced228ce46""...",LOCAL,RandomForestClassifier,run_sklearn.py,None,None
137,70edaa0383064c6cadd16ce4e50ab888,12,FINISHED,file:///home/zeyusun/work/flare-prediction-sma...,2021-05-25 19:57:10.963000+00:00,2021-05-25 19:57:13.730000+00:00,0.882353,1.000000,0.882353,0.971330,...,M_QS_6hr_sharp_balanced_SGDClassifier,True,a59f93c4de3e14b2a4037fbe9146138c6bdc1fd1,81d5aeb899f444c2998491ca6bff7ab9,"[{""run_id"": ""70edaa0383064c6cadd16ce4e50ab888""...",LOCAL,SGDClassifier,run_sklearn.py,None,None
141,15eb7e41f03640098a39b57a536869de,12,FINISHED,file:///home/zeyusun/work/flare-prediction-sma...,2021-05-25 19:51:05.103000+00:00,2021-05-25 19:51:15.108000+00:00,0.684834,0.850711,0.684834,0.932431,...,M_QS_6hr_smarp_balanced_HistGradientBoostingCl...,True,a59f93c4de3e14b2a4037fbe9146138c6bdc1fd1,81d5aeb899f444c2998491ca6bff7ab9,"[{""run_id"": ""15eb7e41f03640098a39b57a536869de""...",LOCAL,HistGradientBoostingClassifier,run_sklearn.py,None,None
142,c301b75bb415458ab96242b8d68deef8,12,FINISHED,file

In [251]:
# Specify columns
runs = runs.loc[:, list(columns.keys())]
runs = runs.rename(columns=lambda k: columns[k])

In [258]:
pd.set_option('display.max_rows', 500)
runs.set_index(['database', 'dataset', 'estimator']).unstack(-1).T

database                           M_QS_12hr                     M_QS_24hr  \
dataset                             combined     sharp     smarp  combined   
    estimator                                                                
auc HistGradientBoostingClassifier  0.972940  0.974371  0.926831  0.950536   
    RandomForestClassifier          0.953481  0.960326  0.904965  0.940440   
    SGDClassifier                   0.961425  0.970559  0.923108  0.941003   
tss HistGradientBoostingClassifier  0.909091  0.899522  0.695828  0.819444   
    RandomForestClassifier          0.827751  0.846890  0.647376  0.780556   
    SGDClassifier                   0.861244  0.885167  0.679677  0.772222   

database                                               M_QS_48hr            \
dataset                                sharp     smarp  combined     sharp   
    estimator                                                                
auc HistGradientBoostingClassifier  0.955845  0.925274  0.922889  0.925681   
    RandomForestClassifier          0.951539  0.909318  0.908423  0.917685   
    SGDClassifier                   0.952801  0.924950  0.903670  0.916553   
tss HistGradientBoostingClassifier  0.816667  0.697138  0.753695  0.753695   
    RandomForestClassifier          0.816667  0.674086  0.711002  0.724138   
    SGDClassifier                   0.808333  0.692369  0.696223  0.735632   

database                                      M_QS_6hr  ...  M_Q_12hr  \
dataset                                smarp  combined  ...     smarp   
    estimator                                           ...             
auc HistGradientBoostingClassifier  0.907943  0.973448  ...  0.981650   
    RandomForestClassifier          0.886000  0.976555  ...  0.974081   
    SGDClassifier                   0.901468  0.968011  ...  0.982515   
tss HistGradientBoostingClassifier  0.649167  0.907563  ...  0.870455   
    RandomForestClassifier          0.622918  0.865546  ...  0.829545   
    SGDClassifier                   0.639071  0.873950  ...  0.886364   

database                            M_Q_24hr                      M_Q_48hr  \
dataset                             combined     sharp     smarp  combined   
    estimator                                                                
auc HistGradientBoostingClassifier  0.989316  0.990391  0.974357  0.987543   
    RandomForestClassifier          0.986906  0.986073  0.962196  0.981530   
    SGDClassifier                   0.986604  0.990642  0.974840  0.982293   
tss HistGradientBoostingClassifier  0.903790  0.900875  0.829596  0.896400   
    RandomForestClassifier          0.880466  0.880466  0.818386  0.861282   
    SGDClassifier                   0.903790  0.892128  0.831839  0.885865   

database                                                 M_Q_6hr            \
dataset                                sharp     smarp  combined     sharp   
    estimator                                                                
auc HistGradientBoostingClassifier  0.989229  0.964156  0.988667  0.978976   
    RandomForestClassifier          0.985241  0.948221  0.973157  0.974187   
    SGDClassifier                   0.989256  0.965374  0.980731  0.990087   
tss HistGradientBoostingClassifier  0.910448  0.805109  0.888060  0.895522   
    RandomForestClassifier          0.896400  0.751095  0.880597  0.895522   
    SGDClassifier                   0.909570  0.815328  0.880597  0.895522   

database                                      
dataset                                smarp  
    estimator                                 
auc HistGradientBoostingClassifier  0.948075  
    RandomForestClassifier          0.933706  
    SGDClassifier                   0.946261  
tss HistGradientBoostingClassifier  0.767717  
    RandomForestClassifier          0.736220  
    SGDClassifier                   0.759843  

[6 rows x 24 columns]

In [259]:
# Cosmetic changes:
extract_hours = lambda s: s.str.split('_').str[2].str.replace('hr', '').astype(int)

# df_latex = (df
#     .rename(columns=lambda k: k.split('.')[1])
#     .set_index(['database_name', 'dataset_name', 'estimator_name'])
#     .unstack(-1).T
# )

# arnet
df = (runs
    #.dropna()
    #.rename(columns=lambda k: columns[k])
    .set_index(['database', 'dataset', 'estimator'])
    .unstack(-1).T
    #.sort_values('database', axis=1, key=extract_hours)
    .round(3)
)
df

database                           M_QS_12hr               M_QS_24hr         \
dataset                             combined  sharp  smarp  combined  sharp   
    estimator                                                                 
auc HistGradientBoostingClassifier     0.973  0.974  0.927     0.951  0.956   
    RandomForestClassifier             0.953  0.960  0.905     0.940  0.952   
    SGDClassifier                      0.961  0.971  0.923     0.941  0.953   
tss HistGradientBoostingClassifier     0.909  0.900  0.696     0.819  0.817   
    RandomForestClassifier             0.828  0.847  0.647     0.781  0.817   
    SGDClassifier                      0.861  0.885  0.680     0.772  0.808   

database                                  M_QS_48hr               M_QS_6hr  \
dataset                             smarp  combined  sharp  smarp combined   
    estimator                                                                
auc HistGradientBoostingClassifier  0.925     0.923  0.926  0.908    0.973   
    RandomForestClassifier          0.909     0.908  0.918  0.886    0.977   
    SGDClassifier                   0.925     0.904  0.917  0.901    0.968   
tss HistGradientBoostingClassifier  0.697     0.754  0.754  0.649    0.908   
    RandomForestClassifier          0.674     0.711  0.724  0.623    0.866   
    SGDClassifier                   0.692     0.696  0.736  0.639    0.874   

database                            ... M_Q_12hr M_Q_24hr                \
dataset                             ...    smarp combined  sharp  smarp   
    estimator                       ...                                   
auc HistGradientBoostingClassifier  ...    0.982    0.989  0.990  0.974   
    RandomForestClassifier          ...    0.974    0.987  0.986  0.962   
    SGDClassifier                   ...    0.983    0.987  0.991  0.975   
tss HistGradientBoostingClassifier  ...    0.870    0.904  0.901  0.830   
    RandomForestClassifier          ...    0.830    0.880  0.880  0.818   
    SGDClassifier                   ...    0.886    0.904  0.892  0.832   

database                           M_Q_48hr                M_Q_6hr         \
dataset                            combined  sharp  smarp combined  sharp   
    estimator                                                               
auc HistGradientBoostingClassifier    0.988  0.989  0.964    0.989  0.979   
    RandomForestClassifier            0.982  0.985  0.948    0.973  0.974   
    SGDClassifier                     0.982  0.989  0.965    0.981  0.990   
tss HistGradientBoostingClassifier    0.896  0.910  0.805    0.888  0.896   
    RandomForestClassifier            0.861  0.896  0.751    0.881  0.896   
    SGDClassifier                     0.886  0.910  0.815    0.881  0.896   

database                                   
dataset                             smarp  
    estimator                              
auc HistGradientBoostingClassifier  0.948  
    RandomForestClassifier          0.934  
    SGDClassifier                   0.946  
tss HistGradientBoostingClassifier  0.768  
    RandomForestClassifier          0.736  
    SGDClassifier                   0.760  

[6 rows x 24 columns]

In [164]:
df_style = (df
            .style
            .background_gradient(axis=None)#, vmin=0.7)
            .set_precision(3))
df_style

## Functions

In [179]:
def get_columns(name):
    tags = {
        'tags.database_name': 'database',
        'tags.dataset_name': 'dataset',
        #'tags.balanced': 'balanced',
        'tags.estimator_name': 'estimator',
    }
    if name == 'arnet':
        metrics = {'metrics.test/' + m: m for m in [
            'auc',
            'tss',
            #'hss2',
            #'precision',
            #'recall',
        ]}
    elif name == 'sklearn':
        metrics = {'metrics.' + m: new_m for m, new_m in [
            ['auc', 'auc'],
            ['tss_opt', 'tss'],
            #'tss',
            #'hss2',
            #'precision',
            #'recall',
        ]}
    else:
        raise
    columns = {**tags, **metrics}
    return columns

In [223]:
def retrieve_runs(experiment_name, parent_run_name, package_name):
    # Specify experiment
    exp_id = client.get_experiment_by_name(experiment_name).experiment_id
    runs = mlflow.search_runs(exp_id)
    #runs = mlflow.search_runs(“<experiment_id>”, “metrics.loss < 2.5”)
    #runs[['params.num_epochs','metrics.loss']].groupby("params.num_epochs").agg("min")

    # Specify parent run
    parentRunId = runs.loc[runs['tags.mlflow.runName'] == parent_run_name, 'run_id'].item()
    runs = runs.loc[runs['tags.mlflow.parentRunId'] == parentRunId]

    # Specify rows
    runs.loc[runs['status'] == 'FINISHED']
    #runs = runs.loc[(runs['tags.estimator_name'] != 'HistGradientBoostingClassifier')]
    runs = runs.loc[runs['tags.balanced'] == 'True']

    # Specify columns
    columns = get_columns(package_name)
    runs = runs.loc[:, list(columns.keys())]

    # Cosmetics
    runs = runs.rename(columns=lambda k: columns[k])
    runs.loc[runs['dataset'] == 'combined', 'dataset'] = 'fused'
    estimator_names = {
        'HistGradientBoostingClassifier': 'HGB',
        'RandomForestClassifier': 'RF',
        'SGDClassifier': 'LG',
    }
    mask = runs['estimator'].isin(estimator_names)
    runs.loc[mask, 'estimator'] = runs.loc[mask, 'estimator'].map(estimator_names)

    metric_names = {
        'auc': 'AUC',
        'tss': 'TSS',
    }
    runs = runs.rename(columns=metric_names)
    return runs

In [224]:
def cosmetic_change(runs):

    # Cosmetic changes:
    extract_hours = lambda s: s.str.split('_').str[2].str.replace('hr', '').astype(int)

    # df_latex = (df
    #     .rename(columns=lambda k: k.split('.')[1])
    #     .set_index(['database_name', 'dataset_name', 'estimator_name'])
    #     .unstack(-1).T
    # )

    # arnet
    df = (runs
        #.dropna()
        #.rename(columns=lambda k: columns[k])
        .set_index(['database', 'dataset', 'estimator'])
        .unstack(-1).T
        .sort_values('database', axis=1, key=extract_hours)
        .round(3)
    )

    return df

def style_change(df):
    df_style = (df
            .style
            .background_gradient(axis=None)#, vmin=0.7)
            .set_precision(3))
    return df_style

In [235]:
experiment_name = 'leaderboard1'
runs_dict = {}
runs_dict['sklearn'] = retrieve_runs(experiment_name, 'sklearn_balanced', 'sklearn')
runs_dict['arnet'] = retrieve_runs(experiment_name, 'arnet_balanced', 'arnet')
runs_dict['sklearn']

,database,dataset,estimator,AUC,TSS
3,M_Q_6hr,fused,HGB,0.988667,0.888060
4,M_Q_6hr,fused,RF,0.973157,0.880597
5,M_Q_6hr,fused,LG,0.980731,0.880597
9,M_Q_6hr,sharp,HGB,0.978976,0.895522
10,M_Q_6hr,sharp,RF,0.974187,0.895522
...,...,...,...,...,...
136,M_QS_6hr,sharp,RF,0.976343,0.857143
137,M_QS_6hr,sharp,LG,0.971330,0.890756
141,M_QS_6hr,smarp,HGB,0.932431,0.699052
142,M_QS_6hr,smarp,RF,0.922141,0.668246


In [236]:
dfs = {}
dfs['sklearn'] = cosmetic_change(runs_dict['sklearn'])
dfs['arnet'] = cosmetic_change(runs_dict['arnet'])

In [234]:
df_latex = dfs['sklearn'][['M_Q_12hr', 'M_Q_24hr']]
print(df_latex.to_latex(
    #column_format='cccc',
    multicolumn_format='c'))
df_latex

\begin{tabular}{llrrrrrr}
\toprule
    & database & \multicolumn{3}{c}{M\_Q\_12hr} & \multicolumn{3}{c}{M\_Q\_24hr} \\
    & dataset &    fused &  sharp &  smarp &    fused &  sharp &  smarp \\
{} & estimator &          &        &        &          &        &        \\
\midrule
AUC & HGB &    0.995 &  0.996 &  0.982 &    0.989 &  0.990 &  0.974 \\
    & LG &    0.995 &  0.997 &  0.983 &    0.987 &  0.991 &  0.975 \\
    & RF &    0.993 &  0.995 &  0.974 &    0.987 &  0.986 &  0.962 \\
TSS & HGB &    0.967 &  0.960 &  0.870 &    0.904 &  0.901 &  0.830 \\
    & LG &    0.962 &  0.974 &  0.886 &    0.904 &  0.892 &  0.832 \\
    & RF &    0.938 &  0.950 &  0.830 &    0.880 &  0.880 &  0.818 \\
\bottomrule
\end{tabular}



database      M_Q_12hr               M_Q_24hr              
dataset          fused  sharp  smarp    fused  sharp  smarp
    estimator                                              
AUC HGB          0.995  0.996  0.982    0.989  0.990  0.974
    LG           0.995  0.997  0.983    0.987  0.991  0.975
    RF           0.993  0.995  0.974    0.987  0.986  0.962
TSS HGB          0.967  0.960  0.870    0.904  0.901  0.830
    LG           0.962  0.974  0.886    0.904  0.892  0.832
    RF           0.938  0.950  0.830    0.880  0.880  0.818

In [240]:
style_change(dfs['sklearn'])

## Combining sklearn and arnet reports

In [200]:
runs_comb = pd.concat(runs_dict.values())
df_comb = cosmetic_change(runs_comb)
df_comb = df_comb[('auc', ('C3D', 'MLP'))]

style_change(df_comb)

KeyError: ('auc', ('C3D', 'MLP'))